In [ ]:
%pip install geopandas scipy

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import numpy as np
import plotly.io as pio

import os
import sys
sys.path.append(os.path.join(os.path.abspath(".."), "functions"))

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

PROJECT_NAME = "AreaVerde"
project = dh.get_or_create_project(PROJECT_NAME)

# Choice of the spiras pseudo-automatic

## Load data

In [ ]:
import data_reader
import spatial_utils
import road_network

In [ ]:
# Load AV
gdf_AreaVerde = data_reader.AV_shape(namefile="area_verde_manual_v1.geojson", datapath="data")

# Create a buffer around the border
out_boundary = gdf_AreaVerde.copy()
out_boundary['geometry'] = spatial_utils.buffer_around(gdf=out_boundary, buffer_size=400)
in_boundary = gdf_AreaVerde.copy()
in_boundary['geometry'] = spatial_utils.buffer_around(gdf=in_boundary, buffer_size=-100)
buffered_boundary = out_boundary.difference(in_boundary)

# Create a buffered AV
buffered_AreaVerde = gdf_AreaVerde.copy() 
buffered_AreaVerde['geometry'] = spatial_utils.buffer_around(gdf=buffered_AreaVerde, buffer_size=2000)

# Load road data
if os.path.exists(f"data/results/geo_edges_v5.geojson") and os.path.exists(f"data/results/geo_nodes_v5.geojson"):
    edges, nodes = data_reader.road_data(edges_namefile="geo_edges_v5.geojson", nodes_namefile="geo_nodes_v5.geojson", datapath="data/results")
else:
    edges, nodes = road_network.create_road_data(
        df_BAV=buffered_AreaVerde,
        relevant_highway=True, connected_network=True,
        edges_namefile="geo_edges_v5.geojson", nodes_namefile="geo_nodes_v5.geojson", datapath="data/results",
    )

# Load spira data
spira = data_reader.spira_shapes(namefile="SpiraFlowData.parquet", datapath="data", project=project)
spira_codes = data_reader.spira_codes(namefile="SpiraFlowData.parquet", datapath="data", project=project)

## Transform the data
### Spira and roads

In [ ]:
import spatial_assignment
import spatial_utils
from constants import CRS_PROJECTED

In [ ]:
# Find the edges that touches a portion of the buffered border
filtered_edges = []
for idx, edge in edges.iterrows():
    if buffered_boundary.intersects(edge.geometry).any():
        filtered_edges.append(edge['link_id'])

edges = (
    edges
    .assign(id_zone=edges['link_id']
            .isin(filtered_edges)
            .apply(lambda x: 'Inside' if x else 'Outside'))
)

In [ ]:
# Assign spiras to roads
spira = spatial_assignment.spira_to_road(df_spira=spira, df_edge=edges)

In [ ]:
# Filter the edges that are on the border and that are important roads, then filter the spiras
edges = edges[(edges['id_zone']=='Inside') & (edges['highway_ok']==True)].reset_index(drop=True)
spira = spira[(spira['id_zone']=='Inside') & (spira['highway_ok']==True)].reset_index(drop=True)

In [ ]:
# Check the distance from the spiras to the corresponding road and to the border
spira['distances_from_roads'] = spira.apply(lambda row: row['geometry_x'].distance(row['geometry_y']), axis=1)
spira['distances_from_areaverde'] = spira.apply(lambda row: row['geometry_x'].distance(buffered_boundary), axis=1)

# Filter the spira based on the distance
buffer_fr = 10 #buffer from roads
buffer_fa = 150 #buffer from areaverde
spira = spira[(spira['distances_from_roads'] < buffer_fr) & (spira['distances_from_areaverde']  < buffer_fa)].drop(columns=['distances_from_roads', 'distances_from_areaverde'])
print(f"There is a total of {spira.shape[0]} spiras")

In [ ]:
# Get the direction of the road with respect to the boundary
edges['linestring_direction'] = edges.apply(lambda row: spatial_utils.direction_wrt_polygon(row['geometry'], polygon=gdf_AreaVerde.geometry.iloc[0]), axis=1)

spira = (
    spira
    .merge(edges[['u', 'v', 'key', 'linestring_direction']], on=['u', 'v', 'key'], how='left')
    .rename(columns={'linestring_direction': 'spira_direction'})
)

# Spira flows and accuracy

In [ ]:
spira_code_directions = pd.merge(spira[['spira_unique_id', 'spira_direction']], spira_codes)
spire_in = spira_code_directions['spira_code'][spira_code_directions['spira_direction'] == 'incoming'].to_list()
spire_out = spira_code_directions['spira_code'][spira_code_directions['spira_direction'] == 'outgoing'].to_list()

In [ ]:
# Load and filter spira accuracy
sel5macc = [[('spira_code', '==', id)] for id in spire_in + spire_out]
accuracy = data_reader.spira_accuracy(namefile="SpiraAccuracy.parquet", datapath="data", project=project, filters=sel5macc)

# Filter the data within a time interval
start_date = datetime(2024, 6, 1 ,0 , 0)
end_date = datetime(2024, 7, 31, 23, 55)

accuracy = accuracy[accuracy['DateTime'].between(start_date, end_date, inclusive='left')]
accuracy = accuracy.sort_values('DateTime')

In [ ]:
# Compute some statistics
df = accuracy.groupby('spira_code').describe(percentiles=[.1,.25,.5,.75,.9], include=[np.number]).reset_index()
df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
df = df.rename(columns={'spira_code_': 'spira_code'})
df = df.merge(spira_code_directions, on='spira_code') #add direction

In [ ]:
# Print some statistics
percentile_cols = [col for col in df.columns if '%' in col] 

print(f"# incoming spiras: {df[df['spira_direction']=='incoming'].shape[0]}")
print(f"# spiras with 10% percentile less than 100: {df[(df['count_10%']<100)&(df['spira_direction']=='incoming')].shape[0]}, that is {round(df[(df['count_10%']<100)&(df['spira_direction']=='incoming')].shape[0]/df[df['spira_direction']=='incoming'].shape[0]*100)}% of all spiras")
print(f"# spiras with 25% percentile less than 100: {df[(df['count_25%']<100)&(df['spira_direction']=='incoming')].shape[0]}, that is {round(df[(df['count_25%']<100)&(df['spira_direction']=='incoming')].shape[0]/df[df['spira_direction']=='incoming'].shape[0]*100)}% of all spiras")
print(f"# spiras with 50% percentile less than 100: {df[(df['count_50%']<100)&(df['spira_direction']=='incoming')].shape[0]}, that is {round(df[(df['count_50%']<100)&(df['spira_direction']=='incoming')].shape[0]/df[df['spira_direction']=='incoming'].shape[0]*100)}% of all spiras")
print(f"# spiras with 75% percentile less than 100: {df[(df['count_75%']<100)&(df['spira_direction']=='incoming')].shape[0]}, that is {round(df[(df['count_75%']<100)&(df['spira_direction']=='incoming')].shape[0]/df[df['spira_direction']=='incoming'].shape[0]*100)}% of all spiras")
print(f"# spiras with 90% percentile less than 100: {df[(df['count_90%']<100)&(df['spira_direction']=='incoming')].shape[0]}, that is {round(df[(df['count_90%']<100)&(df['spira_direction']=='incoming')].shape[0]/df[df['spira_direction']=='incoming'].shape[0]*100)}% of all spiras")

print(f"# outgoing spiras: {df[df['spira_direction']=='outgoing'].shape[0]}")
print(f"# spiras with 10% percentile less than 100: {df[(df['count_10%']<100)&(df['spira_direction']=='outgoing')].shape[0]}, that is {round(df[(df['count_10%']<100)&(df['spira_direction']=='outgoing')].shape[0]/df[df['spira_direction']=='outgoing'].shape[0]*100)}% of all spiras")
print(f"# spiras with 25% percentile less than 100: {df[(df['count_25%']<100)&(df['spira_direction']=='outgoing')].shape[0]}, that is {round(df[(df['count_25%']<100)&(df['spira_direction']=='outgoing')].shape[0]/df[df['spira_direction']=='outgoing'].shape[0]*100)}% of all spiras")
print(f"# spiras with 50% percentile less than 100: {df[(df['count_50%']<100)&(df['spira_direction']=='outgoing')].shape[0]}, that is {round(df[(df['count_50%']<100)&(df['spira_direction']=='outgoing')].shape[0]/df[df['spira_direction']=='outgoing'].shape[0]*100)}% of all spiras")
print(f"# spiras with 75% percentile less than 100: {df[(df['count_75%']<100)&(df['spira_direction']=='outgoing')].shape[0]}, that is {round(df[(df['count_75%']<100)&(df['spira_direction']=='outgoing')].shape[0]/df[df['spira_direction']=='outgoing'].shape[0]*100)}% of all spiras")
print(f"# spiras with 90% percentile less than 100: {df[(df['count_90%']<100)&(df['spira_direction']=='outgoing')].shape[0]}, that is {round(df[(df['count_90%']<100)&(df['spira_direction']=='outgoing')].shape[0]/df[df['spira_direction']=='outgoing'].shape[0]*100)}% of all spiras")

print(f"# spiras: {df.shape[0]}")
print(f"# spiras with 10% percentile less than 100: {df[df['count_10%']<100].shape[0]}, that is {round(df[df['count_10%']<100].shape[0]/df.shape[0]*100)}% of all spiras")
print(f"# spiras with 25% percentile less than 100: {df[df['count_25%']<100].shape[0]}, that is {round(df[df['count_25%']<100].shape[0]/df.shape[0]*100)}% of all spiras")
print(f"# spiras with 50% percentile less than 100: {df[df['count_50%']<100].shape[0]}, that is {round(df[df['count_50%']<100].shape[0]/df.shape[0]*100)}% of all spiras")
print(f"# spiras with 75% percentile less than 100: {df[df['count_75%']<100].shape[0]}, that is {round(df[df['count_75%']<100].shape[0]/df.shape[0]*100)}% of all spiras")
print(f"# spiras with 90% percentile less than 100: {df[df['count_90%']<100].shape[0]}, that is {round(df[df['count_90%']<100].shape[0]/df.shape[0]*100)}% of all spiras")

In [ ]:
# Select the spira that have a 10% percentile of accuracy equal to 100
# (meaning that only 10% of the measurements have accuracy < 100)
spira_in_ok_codes =  df['spira_code'][(df['spira_direction']=='incoming') & (df['count_10%']>=100)].to_list()
print(len(spira_in_ok_codes))

spire_out_ok_codes = df['spira_code'][(df['spira_direction']=='outgoing') & (df['count_10%']>=100)].to_list()
print(len(spire_out_ok_codes))

### In and out flows

In [ ]:
# Load 5min spiras
sel5m = [[('sensor_id', '==', id)] for id in spira_in_ok_codes + spire_out_ok_codes]
spira5m = data_reader.spira_flows(namefile="Spira5mFlowData.parquet", datapath="data", project=project, filters=sel5m)

spira5m = spira5m[spira5m['DateTime'].between(start_date, end_date, inclusive='left')]
spira5m = pd.merge(spira5m, spira_codes, left_on='sensor_id', right_on='spira_code', how='left').drop(columns=['sensor_id'])
spira5m = spira5m.sort_values('DateTime')

In [ ]:
# Filter the data measured during weekdays (or the day of interest)
import temporal_utils

holiday_namefile = "data/holiday_list.csv"
holiday_list = pd.read_csv(holiday_namefile, header=None)[0].to_list()

is_week = spira5m['DateTime'].apply(
    lambda x: temporal_utils.add_daytype(x, holiday_list=holiday_list)
    ) == "Weekday" 
spira5m = spira5m[is_week]

In [ ]:
# inflow from gate estimation (vehicles/hour)
# computed averaging both working days and weekend-days
in_flow_all = [
    2234.        ,  2144.06748188,  2056.54693675,  1971.43836461,    1888.74176547,  1808.45713931,  1730.58448615,  1655.12380598,
    1582.0750988 ,  1511.43836461,  1443.21360342,  1377.40081521,    1314.        ,  1253.01115778,  1194.43428855,  1138.26939231,
    1084.51646907,  1033.17551882,   984.24654156,   937.74831199,     893.69960481,   852.10042002,   812.95075762,   776.25061762,
     742.        ,   710.19890478,   680.84733194,   653.9452815 ,     629.49275345,   607.48974779,   587.93626452,   570.49743322,
     554.83838348,   540.95911528,   528.85962864,   518.53992354,     510.        ,   503.23985801,   498.25949757,   495.05891868,
     493.63812134,   493.99710556,   496.13587132,   500.26708868,     506.60342768,   515.1448883 ,   525.89147057,   538.84317447,
     554.        ,   571.36194717,   590.92901597,   612.70120641,     636.67851849,   662.86095219,   691.24850754,   724.03892358,     
     763.42993936,   809.4215549 ,   862.01377018,   921.20658522,     987.        ,  1059.39401453,  1138.38862882,  1223.98384285,
    1316.17965663,  1414.97607016,  1520.37308344,  1639.0827032 ,    1777.81693616,  1936.57578232,  2115.35924168,  2314.16731424,
    2533.        ,  2771.85729896,  3030.73921113,  3309.64573649,    3608.57687506,  3927.53262683,  4266.51299179,  4609.742635  ,
    4941.44622147,  5261.6237512 ,  5570.2752242 ,  5867.40064047,    6153.        ,  6427.0733028 ,  6689.62054886,  6940.6417382 ,
    7180.13687079,  7408.10594666,  7624.54896579,  7821.9892646 ,    7992.9501795 ,  8137.43171049,  8255.43385757,  8346.95662074,
    8412.        ,  8450.56399535,  8462.6486068 ,  8448.25383433,    8407.37967796,  8340.02613767,  8246.19321348,  8143.32177742,
    8048.85270156,  7962.78598589,  7885.1216304 ,  7815.85963511,    7755.        ,  7702.54272508,  7658.48781036,  7622.83525582,
    7595.58506147,  7576.73722731,  7566.29175335,  7561.30229308,    7558.82250003,  7558.8523742 ,  7561.39191558,  7566.44112418,
    7574.        ,  7584.06854303,  7596.64675328,  7611.73463075,    7629.33217543,  7649.43938733,  7672.05626645,  7696.25335298,
    7721.10118714,  7746.59976892,  7772.74909832,  7799.54917535,    7827.        ,  7855.10157227,  7883.85389217,  7913.25695969,
    7943.31077483,  7974.0153376 ,  8005.37064798,  8031.81647793,    8047.79259936,  8053.29901229,  8048.3357167 ,  8032.90271261,
    8007.        ,  7970.62757888,  7923.78544926,  7866.47361112,    7798.69206448,  7720.44080932,  7631.71984565,  7541.32000167,
    7458.03210557,  7381.85615736,  7312.79215702,  7250.84010457,    7196.        ,  7148.27184331,  7107.6556345 ,  7074.15137358,
    7047.75906054,  7028.47869538,  7016.3102781 ,  7008.1801787 ,    7001.01476719,  6994.81404357,  6989.57800783,  6985.30665997,
    6982.        ,  6979.65802791,  6978.28074371,  6977.8681474 ,    6978.42023896,  6979.93701842,  6982.41848576,  6989.40448166,
    7004.43484682,  7027.50958124,  7058.62868491,  7097.79215783,    7145.        ,  7200.25221143,  7263.54879211,  7334.88974204,
    7414.27506123,  7501.70474967,  7597.17880737,  7701.47626465,    7815.37615187,  7938.87846901,  8071.98321608,  8214.69039307,
    8367.        ,  8528.91203685,  8700.42650364,  8881.54340035,    9072.26272698,  9272.58448355,  9482.50867004,  9685.34904154,
    9864.4193531 , 10019.71960472, 10151.24979641, 10259.00992817,   10343.        , 10403.22001189, 10439.66996385, 10452.34985588,
   10441.25968798, 10406.39946014, 10347.76917236, 10277.54059724,   10207.88550734, 10138.80390267, 10070.29578322, 10002.361149  ,
    9935.        ,  9868.21233623,  9801.99815769,  9736.35746437,    9671.29025627,  9606.79653341,  9542.87629577,  9473.76848614,
    9393.71204731,  9302.70697928,  9200.75328206,  9087.85095563,    8964.        ,  8829.20041517,  8683.45220114,  8526.75535792,
    8359.10988549,  8180.51578386,  7990.97305303,  7795.20959703,    7597.9533199 ,  7399.20422163,  7198.96230222,  6997.22756168,
    6794.        ,  6589.27961718,  6383.06641323,  6175.36038814,    5966.16154191,  5755.46987455,  5543.28538605,  5336.47393166,
    5141.90136662,  4959.56769093,  4789.4729046 ,  4631.61700762,    4486.        ,  4352.62188173,  4231.48265281,  4122.58231325,
    4025.92086304,  3941.49830218,  3869.31463068,  3805.11347969,    3744.63848039,  3687.88963277,  3634.86693683,  3585.57039257,
    3540.        ,  3498.15575911,  3460.0376699 ,  3425.64573237,    3394.97994652,  3368.04031236,  3344.82682988,  3321.23407907,
    3293.15663993,  3260.59451245,  3223.54769664,  3182.01619249,    3136.        ,  3085.49911918,  3030.51355002,  2971.04329253,
    2907.0883467 ,  2838.64871254,  2765.72439004,  2688.31537921,    2606.42168004,  2520.04329253,  2429.18021669,  2333.83245251]

In [ ]:
# inflow from gate estimation (vehicles/hour) computed for different daytypes
in_flow_week = np.array([ 
    221.25 , 208.70090169, 196.59507377, 184.93251626,173.71322915, 162.93721244, 152.60446612, 142.71499021,
    133.2687847 , 124.26584959, 115.70618488, 107.58979057,99.91666667, 92.68681316, 85.90023005, 79.55691735,
    73.65687504, 68.20010313, 63.18660163, 58.57940003,54.34152786, 50.4729851 , 46.97377176, 43.84388784,
    41.08333333, 38.69210825, 36.67021258, 35.01764633,33.73440951, 32.82050209, 32.2759241 , 31.97296144,
    31.78390001, 31.70873982, 31.74748086, 31.90012315,32.16666667, 32.54711142, 33.04145742, 33.64970465,
    34.37185312, 35.20790282, 36.15785376, 37.26801653,38.58470172, 40.10790933, 41.83763935, 43.7738918 ,
    45.91666667, 48.26596395, 50.82178366, 53.58412578,56.55299032, 59.72837728, 63.11028667, 67.02310605,
    71.79122301, 77.41463755, 83.89334968, 91.22735938,99.41666667, 108.46127153, 118.36117398, 129.11637401,
    140.72687162, 153.19266681, 166.51375958, 181.49843978,198.95499726, 218.88343202, 241.28374407, 266.15593339,
    293.5 , 323.31594389, 355.60376506, 390.36346351,427.59503924, 467.29849225, 509.47382255, 552.31727381,
    594.02508973, 634.59727031, 674.03381555, 712.33472545,749.5 , 785.52963921, 820.42364308, 854.18201161,886.80474479, 918.29184263, 948.64330513, 976.28773216,
    999.65372359, 1018.74127943, 1033.55039966, 1044.0810843 ,1050.33333333, 1052.30714677, 1050.00252461, 1043.41946686,
    1032.5579735 , 1017.41804455, 997.99967999, 977.02114803,957.20071686, 938.53838647, 921.03415686, 904.68802804,
    889.5 , 875.47007275, 862.59824628, 850.88452059,840.32889569, 830.93137158, 822.69194825, 815.37241668,
    808.73456786, 802.77840178, 797.50391844, 792.91111785,
    789. , 785.77056489, 783.22281253, 781.35674292,780.17235604, 779.66965191, 779.84863052, 780.46888892,
    781.29002415, 782.3120362 , 783.53492508, 784.9586908 ,786.58333333, 788.4088527 , 790.43524889, 792.66252192,
    795.09067177, 797.71969844, 800.54960195, 803.02721276,804.59936133, 805.26604768, 805.02727179, 803.88303368,
    801.83333333, 798.87817076, 795.01754596, 790.25145892,784.57990966, 778.00289817, 770.52042444, 763.0159827 ,
    756.37306713, 750.59167774, 745.67181454, 741.61347751,738.41666667, 736.081382 , 734.60762352, 733.99539122,
    734.24468509, 735.35550515, 737.32785139, 739.61300218,741.6622359 , 743.47555253, 745.0529521 , 746.39443459,
    747.5 , 748.36964834, 749.0033796 , 749.40119378,749.56309089, 749.48907093, 749.17913389, 749.28980052,
    750.47759157, 752.74250705, 756.08454694, 760.50371126,766. , 772.57341316, 780.22395074, 788.95161274,
    798.75639917, 809.63831001, 821.59734528, 834.56717618,848.48147392, 863.34023851, 879.14346994, 895.89116821,
    913.58333333, 932.2199653 , 951.8010641 , 972.32662975,993.79666225, 1016.21116159, 1039.57012777, 1062.26316093,
    1082.67986119, 1100.82022857, 1116.68426305, 1130.27196464,1141.58333333, 1150.61836914, 1157.37707205, 1161.85944207,
    1164.06547919, 1163.99518343, 1161.64855477, 1157.67561751,1152.72639595, 1146.80089008, 1139.89909992, 1132.02102544,
    1123.16666667, 1113.33602359, 1102.5290962 , 1090.74588451,1077.98638852, 1064.25060823, 1049.53854363, 1034.03498586,
    1017.92472607, 1001.20776426, 983.88410042, 965.95373456,947.41666667, 928.27289675, 908.52242481, 888.16525085,
    867.20137486, 845.63079684, 823.4535168 , 800.82280064,777.89191427, 754.66085769, 731.12963089, 707.29823388,
    683.16666667, 658.73492924, 634.0030216 , 608.97094374,583.63869568, 558.0062774 , 532.07368891, 506.86154363,
    483.39045498, 461.66042296, 441.67144757, 423.4235288 ,406.91666667, 392.15086116, 379.12611228, 367.84242003,
    358.29978441, 350.49820541, 344.43768305, 339.53803014,335.21905953, 331.48077121, 328.32316518, 325.74624144,
    323.75 , 322.33444085, 321.49956399, 321.24536942,321.57185714, 322.47902716, 323.96687946, 325.32925698,
    325.86000264, 325.55911644, 324.42659838, 322.46244845,319.66666667, 316.03925302, 311.58020751, 306.28953013,
    300.1672209 , 293.2132798 , 285.42770685, 276.81050203,267.36166534, 257.0811968 , 245.9690964 , 234.02536413
])

in_flow_sun = np.array([281.08333333, 272.21671425, 263.40791479, 254.65693497,
245.96377478, 237.32843422, 228.75091329, 220.231212 ,
211.76933033, 203.3652683 , 195.0190259 , 186.73060314,
178.5 , 170.3272165 , 162.21225263, 154.15510839,
146.15578378, 138.2142788 , 130.33059346, 122.6924307 ,
115.48749348, 108.7157818 , 102.37729566, 96.47203506,
91. , 85.96119048, 81.35560649, 77.18324805,
73.44411514, 70.13820777, 67.26552595, 64.71605564,
62.37978284, 60.25670754, 58.34682974, 56.65014945,
55.16666667, 53.89638138, 52.8392936 , 51.99540333,
51.36471056, 50.94721529, 50.74291753, 50.63392064,
50.50232801, 50.34813963, 50.1713555 , 49.97197562,
49.75 , 49.50542863, 49.23826151, 48.94849864,
48.63614002, 48.30118566, 47.94363555, 47.7466242 ,
47.89328614, 48.38362135, 49.21762984, 50.39531162,
51.91666667, 53.781695 , 55.99039661, 58.5427715 ,
61.43881966, 64.67854111, 68.26193584, 71.98124154,
75.6286959 , 79.20429893, 82.70805062, 86.13995098,
89.5 , 92.78819769, 96.00454404, 99.14903906,
102.22168274, 105.22247508, 108.15141609, 111.24787102,
114.75120513, 118.66141842, 122.97851088, 127.70248252,
132.83333333, 138.37106333, 144.31567249, 150.66716084,
157.42552836, 164.59077506, 172.16290094, 180.05931009,
188.19740664, 196.57719056, 205.19866188, 214.06182058,
223.16666667, 232.51320014, 242.101421 , 251.93132924,
262.00292487, 272.31620789, 282.87117829, 293.45182398,
303.84213283, 314.04210486, 324.05174007, 333.87103845,
343.5 , 352.93862473, 362.18691263, 371.2448637 ,
380.11247795, 388.78975537, 397.27669597, 405.56770901,
413.65720377, 421.54518025, 429.23163845, 436.71657837,
444. , 451.08190335, 457.96228843, 464.64115522,
471.11850372, 477.39433395, 483.4686459 , 489.35951455,
495.08501491, 500.64514696, 506.03991072, 511.26930618,
516.33333333, 521.23199219, 525.96528275, 530.533205 ,
534.93575896, 539.17294462, 543.24476198, 546.37705552,
547.79566974, 547.50060463, 545.49186019, 541.76943643,
536.33333333, 529.18355091, 520.32008916, 509.74294809,
497.45212768, 483.44762795, 467.7294489 , 451.72305971,
436.85392961, 423.12205859, 410.52744665, 399.07009378,
388.75 , 379.5671653 , 371.52158967, 364.61327313,
358.84221567, 354.20841729, 350.71187798, 348.12717879,
346.22890075, 345.01704384, 344.49160809, 344.65259347,
345.5 , 347.03382767, 349.25407649, 352.16074645,
355.75383756, 360.03334981, 364.9992832 , 370.28238605,
375.51340665, 380.69234502, 385.81920114, 390.89397502,
395.91666667, 400.88727607, 405.80580323, 410.67224815,
415.48661083, 420.24889127, 424.95908947, 429.80350492,
434.96843712, 440.45388605, 446.25985174, 452.38633416,
458.83333333, 465.60084925, 472.6888819 , 480.09743131,
487.82649745, 495.87608034, 504.24617997, 512.77852887,
521.31485954, 529.85517199, 538.39946621, 546.94774222,
555.5 , 564.05623956, 572.6164609 , 581.18066401,
589.74884891, 598.32101558, 606.89716402, 615.23828484,
623.10536862, 630.49841536, 637.41742506, 643.86239771,
649.83333333, 655.33023191, 660.35309345, 664.90191795,
668.97670541, 672.57745583, 675.70416921, 678.21305838,
679.96033615, 680.94600253, 681.17005752, 680.63250112,
679.33333333, 677.27255415, 674.45016358, 670.86616161,
666.52054826, 661.41332351, 655.54448737, 648.77734638,
640.97520707, 632.13806944, 622.2659335 , 611.35879924,
599.41666667, 586.43953577, 572.42740656, 557.38027904,
541.29815319, 524.18102903, 506.02890656, 487.93999298,
471.0124955 , 455.24641413, 440.64174887, 427.19849971,
414.91666667, 403.79624973, 393.83724889, 385.03966416,
377.40349554, 370.92874303, 365.61540662, 360.92528836,
356.3201903 , 351.80011243, 347.36505476, 343.01501728,
338.75 , 334.57000291, 330.47502602, 326.46506932,
322.54013282, 318.70021651, 314.9453204 , 311.33466575,
307.92747382, 304.72374461, 301.72347813, 298.92667437,
296.33333333, 293.94345502, 291.75703943, 289.77408657,
287.99459642, 286.41856901, 285.04600431, 283.87690234,
282.91126309, 282.14908657, 281.59037277, 281.23512169])

in_flow_sat = np.array([288.66666667, 278.52872268, 268.57495033, 258.80534961,
249.21992052, 239.81866307, 230.60157725, 221.56866307,
212.71992052, 204.05534961, 195.57495033, 187.27872268,
179.16666667, 171.23878229, 163.49506954, 155.93552843,
148.56015896, 141.36896112, 134.36193491, 127.60331972,
121.15735495, 115.02404059, 109.20337665, 103.69536312,
98.5 , 93.6172873 , 89.047225 , 84.78981312,
80.84505166, 77.21294061, 73.89347997, 70.85540007,
68.06743125, 65.5295735 , 63.24182681, 61.2041912 ,
59.41666667, 57.8792532 , 56.59195081, 55.55475948,
54.76767923, 54.23071005, 53.94385195, 53.87770577,
54.00287237, 54.31935177, 54.82714395, 55.52624891,
56.41666667, 57.4983972 , 58.77144053, 60.23579664,
61.89146554, 63.73844722, 65.77674169, 68.11679125,
70.86903821, 74.03348256, 77.61012431, 81.59896346,
86. , 90.81323394, 96.03866528, 101.67629401,
107.72612014, 114.18814367, 121.06236459, 128.32432452,
135.94956505, 143.93808621, 152.28988797, 161.00497035,
170.08333333, 179.52497693, 189.32990115, 199.49810597,
210.02959141, 220.92435745, 232.18240412, 243.98587277,
256.5169048 , 269.7755002 , 283.76165898, 298.47538114,
313.91666667, 330.08551557, 346.98192785, 364.60590351,
382.95744254, 402.03654494, 421.84321072, 441.51738331,
460.19900615, 477.88807924, 494.58460258, 510.28857617,
525. , 538.71887408, 551.44519841, 563.17897299,
573.92019782, 583.6688729 , 592.42499822, 600.7454755 ,
609.18720643, 617.75019101, 626.43442924, 635.23992113,
644.16666667, 653.21466586, 662.3839187 , 671.6744252 ,
681.08618535, 690.61919915, 700.27346661, 709.70883777,
718.58516269, 726.90244138, 734.66067382, 741.85986003,
748.5 , 754.58109373, 760.10314122, 765.06614248,
769.47009749, 773.31500627, 776.6008688 , 779.58251641,
782.51478038, 785.39766073, 788.23115745, 791.01527054,
793.75 , 796.43534583, 799.07130804, 801.65788661,
804.19508156, 806.68289288, 809.12132057, 810.49745267,
809.79837723, 807.02409424, 802.17460371, 795.24990563,
786.25 , 775.17488683, 762.02456611, 746.79903785,
729.49830204, 710.12235869, 688.67120779, 666.81702682,
646.23199328, 626.91610716, 608.86936847, 592.09177719,
576.58333333, 562.3440369 , 549.37388789, 537.67288629,
527.24103212, 518.07832537, 510.18476605, 503.44066416,
497.72632974, 493.04176278, 489.38696328, 486.76193124,
485.16666667, 484.60116956, 485.06543991, 486.55947772,
489.083283 , 492.63685574, 497.22019594, 502.14084007,
506.70632459, 510.91664952, 514.77181484, 518.27182055,
521.41666667, 524.20635318, 526.64088008, 528.72024738,
530.44445508, 531.81350318, 532.82739167, 534.09623988,
536.23016714, 539.22917345, 543.09325881, 547.82242321,
553.41666667, 559.87598917, 567.20039072, 575.38987131,
584.44443096, 594.36406965, 605.14878739, 615.73310954,
625.05156144, 633.10414311, 639.89085453, 645.41169572,
649.66666667, 652.65576737, 654.37899784, 654.83635807,
654.02784805, 651.9534678 , 648.61321731, 644.92639919,
641.81231605, 639.2709679 , 637.30235473, 635.90647654,
635.08333333, 634.83292511, 635.15525187, 636.05031362,
637.51811035, 639.55864206, 642.17190876, 644.7964583 ,
646.87083855, 648.39504951, 649.36909119, 649.79296357,
649.66666667, 648.99020047, 647.76356499, 645.98676021,
643.65978615, 640.7826428 , 637.35533016, 632.88049917,
626.86080079, 619.29623502, 610.18680185, 599.53250129,
587.33333333, 573.58929798, 558.30039524, 541.4666251 ,
523.08798756, 503.16448263, 481.69611031, 460.29323186,
440.56620854, 422.51504037, 406.13972733, 391.44026943,
378.41666667, 367.06891904, 357.39702656, 349.40098921,
343.080807 , 338.43647993, 335.46800799, 333.55686895,
332.08454055, 331.05102278, 330.45631566, 330.30041918,
330.58333333, 331.30505813, 332.46559357, 334.06493965,
336.10309637, 338.58006373, 341.49584173, 344.29379519,
346.41728892, 347.86632294, 348.64089724, 348.74101181,
348.16666667, 346.9178618 , 344.99459721, 342.3968729 ,
339.12468887, 335.17804512, 330.55694165, 325.26137845,
319.29135554, 312.6468729 , 305.32793054, 297.33452847])

In [ ]:
# Chose the interesting inflow
in_flow = in_flow_week

In [ ]:
# Compute the in-flow from spiras
flow_in = spira5m[spira5m['spira_code'].isin(spire_in)]
flow_in['time'] = flow_in['DateTime'].dt.time
flow_in = flow_in.drop(columns=['spira_unique_id','spira_code','DateTime'])
flow_in = flow_in.groupby('time').sum()

# Compute the out-flow from spiras
flow_out = spira5m[spira5m['spira_code'].isin(spire_out)]
flow_out['time'] = flow_out['DateTime'].dt.time
flow_out = flow_out.drop(columns=['spira_unique_id','spira_code','DateTime'])
flow_out = flow_out.groupby('time').sum()

In [ ]:
flow = pd.merge(flow_in, flow_out, on='time', suffixes=['_in','_out'])
flow = flow / flow.mean()
flow['out-in-ratio'] = flow['count_out'] / flow['count_in']
#flow['count_out'] = flow['count_out'] * len(spire_in_ok)/len(spire_out_ok) ## Given that the in is measured wrt more spiras than the out, expand the second one

flow.plot();

In [ ]:
# Compute the outflow from gates given the inflow from gates and the out-in ratio
raw_out_flow = in_flow * flow['out-in-ratio'].values

# Smooth it
smoothed_out_flow = savgol_filter(raw_out_flow, 12*2+1, 2)

In [ ]:
# Check if the TI=TO during the day
print(sum(in_flow)-sum(smoothed_out_flow)) ##Apparently the if is 2k vehicles higher than the outflow 

# Fix the difference
out_flow = smoothed_out_flow * sum(in_flow) / sum(smoothed_out_flow)

In [ ]:
plt.plot(in_flow, label='inflow')
plt.plot(out_flow, color='red', label='outflow')
plt.xlabel('5min intervals from midnight')
plt.legend()

In [33]:
print(out_flow)

[ 244.77861109  232.36180063  220.31010111  208.62351253  197.30203489
  186.34566818  175.75441241  165.52826759  155.6672337   146.17131075
  137.04049873  128.27479766  119.87420753  111.61712941  103.51764292
   95.421859     88.05247332   81.53910039   75.01201861   69.22820323
   64.40462434   60.34125981   56.69849334   52.96060926   49.75703412
   47.06492426   44.23577288   41.71759661   40.01412602   38.18565376
   36.52013182   35.28771248   34.09765375   33.32988157   32.83798486
   32.69738489   32.13946997   31.79701693   31.23879412   30.93867473
   30.5640021    30.59874219   30.79466099   31.03077592   31.93174691
   33.29475176   34.39689037   35.92930023   37.83949326   39.91341652
   42.39509002   44.81192533   47.5384192    49.71481365   52.61063958
   56.72776874   61.95912839   68.20500193   75.49621639   82.72885491
   91.31445603  101.41624022  110.53830149  119.91744149  128.29367569
  137.3044439   148.01729565  158.65164401  171.00279106  185.37319992
  203.

## Compare with previous results

In [ ]:
# outflow with previous estimation (vehicles/hour)
previous_out_flow = np.array([ 
233.38097382,220.82386919,208.66447564,196.90279317,185.53882178,
174.57256146,164.00401223,153.83317407,144.06004699,134.68463099,
125.70692607,117.12693223,108.94464946,101.0321295,94.13377195,
86.99545139,79.56647229,72.89347297,66.43539924,61.17103286,
57.63701658,53.34527257,50.4033317,47.759661,45.73497488,
43.90603271,41.70428204,39.91449515,39.15679155,38.09222034,
37.59168808,37.72430755,37.48694063,37.97971866,38.86532297,
40.00690705,41.1382239,41.27255557,42.03582926,43.44062896,
45.01849213,47.16432432,49.31850276,52.21059806,54.77757331,
57.14070142,59.22688828,61.83896169,65.21928226,67.04735569,
70.0809006,71.92922162,72.7786066,75.16039375,78.58379158,
83.60695876,89.25750502,93.98382952,102.12187753,108.64578606,
114.67457031,123.2834927,133.47508488,145.54121758,156.08214232,
168.81025967,183.87309895,202.2234539,224.18759278,243.95564543,
265.66429599,292.52932508,317.79647799,344.9508563,378.78430635,
413.33758064,446.84846097,490.01959135,534.3712816,580.88058797,
622.48861554,665.11606446,710.70582349,750.92433028,793.02244353,
833.67680924,867.5229037,900.52273186,930.83950341,960.91070543,
987.83291416,1009.74080772,1026.52101367,1035.17676202,1042.71839831,
1043.99153351,1041.13844737,1035.96755075,1029.41166672,1016.29971021,
1001.3092584,989.47166903,970.10384782,948.5897432,923.5646006,
901.39445702,878.96077706,858.5563716,840.71812307,823.46242241,
810.00473918,800.01069798,790.94707985,783.39820415,778.38441342,
774.31747136,773.38627615,771.95539807,772.00138138,772.89160129,
775.39762518,775.79549931,779.93424535,782.388472,787.14912667,
790.80560406,791.13695772,790.67991276,788.99862582,788.99975721,
789.48802905,794.07434339,795.68325142,796.77293574,799.04201317,
797.54404089,796.20271733,797.73396008,799.33184574,799.70558476,
801.57611639,803.23389929,804.84978994,804.17001722,800.2436378,
799.52714921,803.55718169,801.54781214,800.46789985,797.00742118,
788.25677571,781.47861636,775.77825567,773.19206343,769.4190255,
768.33852595,766.40521157,763.50568329,760.832367,758.60433491,
757.2314636,759.33384749,759.92161156,760.2634731,762.79639775,
762.46683578,761.38786017,758.86695224,756.37666935,757.2691232,
759.48895345,756.83237216,755.9419641,751.60892539,747.80622134,
747.78990459,749.77408703,752.77854767,753.28916998,757.25940755,
764.12894948,769.01993978,779.37591699,795.95227844,811.09899901,
825.35272778,838.29781452,851.89662744,867.06238483,883.20741689,
901.33987243,916.92618364,930.22305078,943.05997435,960.20339662,
978.96899428,993.69348464,1003.04809112,1013.96504438,1022.21601136,
1030.7552496,1038.72961607,1045.85387639,1052.56871626,1056.22471886,
1055.40556205,1061.4493517,1067.53452654,1066.10217331,1062.96188734,
1059.06667558,1055.87584762,1053.14144238,1046.51610526,1039.375427,
1030.81368807,1020.46855986,1013.08636246,1010.64883215,1007.11776449,
997.2943457,986.63314024,979.23301985,971.89403131,964.75315405,
955.17045956,943.69401935,928.20338455,914.7760309,903.36125379,
894.84270558,882.62646182,863.90766424,845.02423897,824.69943747,
805.26726683,781.94129324,756.72007651,732.22047699,706.82035816,
680.49860744,656.96772472,634.47123056,611.51113442,586.81196978,
565.07811412,542.86699149,520.9205191,500.98126772,481.90154566,
464.93236669,448.26999257,435.06428181,421.71897584,409.59461057,
398.61534038,388.06113577,378.86893305,371.13773423,364.25198048,
356.71765738,351.88394111,347.24010377,346.32385715,345.12956257,
346.19381322,346.80991732,346.92820276,346.51040017,348.27544292,
347.80132067,349.39083131,350.65359029,349.07209635,348.27302721,
345.98271159,342.15181282,337.62558026,332.40401389,326.48711372,
319.87487975,312.56731199,304.56441042,295.86617505,286.47260589,
276.38370292,265.59946615,254.11989559
])

In [ ]:
plt.plot(previous_out_flow, color='green', label='previous outflow')
plt.plot(out_flow, color='red', label='current outflow')
plt.xlabel('5min intervals from midnight')
plt.legend()